# INICIALIZAÇÃO

## Instalando as bibliotecas

In [ ]:
# Instalar o Ultralytics
!pip install ultralytics

# Importar a biblioteca
from ultralytics import YOLO

## Montando o drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Salvando os arquivo YAML's de treinamento + validação e teste

In [ ]:
import os

# Definir o caminho da pasta onde o arquivo YAML será salvo
pasta_destino = "/content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/"
os.makedirs(pasta_destino, exist_ok=True)  # Criar a pasta se não existir

# Caminhos do dataset (ajuste conforme necessário)
train_path = "/content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/RTF/train/images"
val_path   = "/content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/RTF/val/images"
test_path  = "/content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/RTF/test/images"

# Conteúdo do arquivo YAML
yaml_content = f"""train: '{train_path}'
val: '{val_path}'

nc: 1  # Número de classes
names: ['obj']
"""

# Caminho completo do arquivo YAML
yaml_train_file = os.path.join(pasta_destino, "dataset_train.yaml")

# Escrever o conteúdo no arquivo
with open(yaml_train_file, "w") as file:
    file.write(yaml_content)

print(f"Arquivo YAML de treino gerado em: {yaml_train_file}")

#-----------------------------------------------------------

# Conteúdo do arquivo YAML
yaml_content = f"""train: '{train_path}'
val: '{test_path}'

nc: 1  # Número de classes
names: ['obj']
"""

# Caminho completo do arquivo YAML
yaml_test_file = os.path.join(pasta_destino, "dataset_test.yaml")

# Escrever o conteúdo no arquivo
with open(yaml_test_file, "w") as file:
    file.write(yaml_content)

print(f"Arquivo YAML de teste gerado em: {yaml_test_file}")


UnicodeEncodeError: 'ascii' codec can't encode character '\xfa' in position 217: ordinal not in range(128)

# TREINAMENTO

## Resetando a GPU

In [ ]:
import torch
torch.cuda.empty_cache()


## Descobrindo o número de núcleos disponíveis

In [ ]:
import multiprocessing

workers_max = multiprocessing.cpu_count()
print(f"Número de núcleos disponíveis: {workers_max}")

Número de núcleos disponíveis: 12


## Treinamento yolo

In [ ]:
# Carregar o modelo YOLOv11x
model = YOLO('yolo11x.pt')

results = model.train(
    data=   '/content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/dataset_train.yaml',
    project='/content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/results',
    name=   'ODC_PN_CENTRAL',
    imgsz=320,
    workers=workers_max - 2,

    iou=0.7,
    conf=0.01,
    patience=50,   # Reduzir a paciência
    epochs=150,    # Reduzir o número de épocas
    batch=64,      # Aumentar o batch size se possível
    max_det=8,
    seed = 5,

    single_cls	=	False,	#Trata todas as classes em conjuntos de dados multi-classe como uma única classe durante o treino.
    #classes = ['pn'],
    plots = True, #Imprime os resultados
    #freeze = 4, #Congela as primeiras camadas convolucionais que tem caracteristicas base para o nosso caso

    flipud = 0.5,
    mosaic=0,    # Reativar mosaic augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.1,
    device='cuda',

    nms=True,
    save_period=True,
    verbose=True,
    scale=0,#0.2,       # Reativar escala com valor moderado
    shear=0,#0.1,       # Reativar cisalhamento com valor moderado
    copy_paste=0,#0.1,  # Reativar copy-paste com valor moderado

    weight_decay=0.0001, # Reativar weight decay
)

Ultralytics 8.3.105 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=detect, mode=train, model=yolo11x.pt, data=/content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/dataset_train.yaml, epochs=150, time=None, patience=50, batch=64, imgsz=320, save=True, save_period=True, cache=False, device=cuda, workers=10, project=/content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/results, name=ODC_PN_CENTRAL5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=5, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=0.01, iou=0.7, max_det=8, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

train: Scanning /content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/CENTRAL/train/labels... 6501 images, 325 backgrounds, 0 corrupt: 100%|██████████| 6501/6501 [01:29<00:00, 72.65it/s] 


train: New cache created: /content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/CENTRAL/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/CENTRAL/val/labels.cache... 1657 images, 16 backgrounds, 0 corrupt: 100%|██████████| 1657/1657 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/results/ODC_PN_CENTRAL5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0001), 173 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 320 train, 320 val
Using 10 dataloader workers
Logging results to /content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/results/ODC_PN_CENTRAL5
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

     59/150      18.8G     0.7424      0.347      1.087         73        320: 100%|██████████| 102/102 [01:17<00:00,  1.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:11<00:00,  1.15it/s]

                   all       1657       3267      0.945       0.88      0.941      0.572
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 9, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



59 epochs completed in 1.495 hours.
Optimizer stripped from /content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/results/ODC_PN_CENTRAL5/weights/last.pt, 114.3MB
Optimizer stripped from /content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/results/ODC_PN_CENTRAL5/weights/best.pt, 114.3MB

Validating /content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/results/ODC_PN_CENTRAL5/weights/best.pt...
Ultralytics 8.3.105 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11x summary (fused): 190 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:13<00:00,  1.01s/it]


                   all       1657       3267      0.935      0.885      0.942        0.6
Speed: 0.0ms preprocess, 3.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/results/ODC_PN_CENTRAL5


# MÉTRICAS

## Tirando as métricas da base de validação

In [ ]:
# Caminho para o modelo treinado
model_path = r'/content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/results/#11 ODC_PN_CENTRAL5/weights/epoch21.pt'

# Carregar o modelo
model = YOLO(model_path)

# Avaliar o modelo usando o arquivo data.yaml
metrics = model.val(data=yaml_train_file, imgsz=320)

NameError: name 'yaml_train_file' is not defined

## Tirando as métricas da base de teste

In [ ]:
# Caminho para o modelo treinado
model_path = r'/content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/results/#5_ODC_PNxBCK_RTF_OVA_1o25_nomos2/weights/epoch38.pt'

yaml_test_file = r'/content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/dataset_test.yaml'
# Carregar o modelo
model = YOLO(model_path)

# Avaliar o modelo usando o arquivo data.yaml
metrics = model.val(data=yaml_test_file, imgsz=320)#, conf = 0.25, iou=0.5)

Ultralytics 8.3.105 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11x summary (fused): 190 layers, 56,828,179 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/RTF/test/labels... 509 images, 210 backgrounds, 0 corrupt: 100%|██████████| 510/510 [00:02<00:00, 251.77it/s]


val: New cache created: /content/drive/MyDrive/Projeto final mestrado/Dados dos treinamentos/Final/OBD/RTF/test/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:04<00:00,  6.59it/s]


                   all        510        579      0.878      0.856      0.885      0.305
Speed: 0.2ms preprocess, 5.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val19
